In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str # 사용자 질문
    answer: str # 세율 계산

    tax_base_equation: str # 과세 표준 게산 수식
    tax_deduction: str # 공제액
    market_ratio: str # 공정시장가액비율
    tax_base: str# 과세표준계산
    
graph_builder = StateGraph(AgentState)

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500, # chunk 하나가 가질 수 있는 token 수
    chunk_overlap = 100 # chunk 간 token 을 겹치게 하는 범위 (유사도 검색의 성능을 더 올림)
)

from langchain_community.document_loaders import TextLoader
text_path = './documents/real_estate_tax.txt'
loader = TextLoader(text_path)
document_list =loader.load_and_split(text_splitter)

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 최초 생성시
embedding_function = OpenAIEmbeddings(model = 'text-embedding-3-large')
vector_store = Chroma.from_documents(
    documents=document_list,
    embedding=embedding_function,
    collection_name= 'real_estate_tax',
    persist_directory='./real_estate_tax_collection'
)

retriever = vector_store.as_retriever(search_kwargs = {'k':3})


# 생성된 retriever 참조시


In [5]:
query = "5억 짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내야하나요?"

In [6]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')

rag_prompt = hub.pull('rlm/rag-prompt')

In [7]:
tax_base_retrieval_chain = (
    {'context': retriever, 'question': RunnablePassthrough()} 
    | rag_prompt 
    | llm 
    | StrOutputParser()
)

tax_base_equation_prompt = ChatPromptTemplate(
    [('system', '사용자의 질문에서 과세표준을 게산하는 방법을 수식으로 나타내주세요. 부연설명 없이 수식만 답변하세요.'),
    ('human', '{tax_base_equation_information}')
    ]
 )

tax_base_equation_chain = (
    {'tax_base_equation_information': RunnablePassthrough()}
    | tax_base_equation_prompt
    | llm
    | StrOutputParser()
)

tax_base_chain = {'tax_base_equation_information' : tax_base_retrieval_chain} | tax_base_equation_chain

def get_tax_base_equation(state:AgentState) -> str:
    tax_base_equation_question = "주택에 대한 종합부동산세 계산시 과세표준을 계산하는 방법을 수식으로 표현해서 알려주세요."
    tax_base_equation = tax_base_chain.invoke(tax_base_equation_question)
    return {'tax_base_equation': tax_base_equation}

In [8]:
get_tax_base_equation({})
# {'tax_base_equation': '과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율'}

{'tax_base_equation': '과세표준 = (총 공시가격 - 공제액) × 공정시장가액비율'}

In [9]:
tax_deduction_chain = (
    {'context': retriever, 'question': RunnablePassthrough()} 
    | rag_prompt 
    | llm 
    | StrOutputParser()
)

def get_tax_deduction(state:AgentState) -> str:
    tax_deduciton_question = "주택에 대한 종합부동산세 계산시 공제금액을 알려주세요"
    tax_deduction = tax_deduction_chain.invoke(tax_deduciton_question)
    return {'tax_deduction': tax_deduction}

In [ ]:
get_tax_deduction({})
# 'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원입니다. 법인 또는 법인으로 보는 단체는 6억 원, 그 외의 경우는 9억 원이 공제됩니다.'

{'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원이며, 그 외의 경우 9억 원입니다.'}

In [11]:
from langchain_community.tools import TavilySearchResults
from datetime import date

tax_market_ratio_prompt = ChatPromptTemplate.from_messages([
    ('system','아래 정보를 기반으로 공정시장 가액비율을 계산해주세요. \n\n Context:\n{context}'),
    ('human','{query}')
])

tavily_search_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

def get_market_ratio(state:AgentState):
    
    query = f'오늘 날짜:({date.today()})에 해당하는 주택 공시가격 공정시장가액비율은 몇%인가요?'
    context = tavily_search_tool.invoke(query)
    
    tax_market_ratio_chain = (
        tax_market_ratio_prompt
        | llm
        | StrOutputParser()
    )
    
    market_ratio = tax_market_ratio_chain.invoke({'context': context, 'query': query})
    
    return {'market_ratio': market_ratio}

In [12]:
get_market_ratio({})
# {'market_ratio': '오늘 날짜인 2025년 7월 14일에 해당하는 주택 공시가격 공정시장가액비율은 다음과 같습니다:\n\n- 공시가격 3억 원 이하: 43%\n- 공시가격 3억 원 초과 ~ 6억 원 이하: 44%\n- 공시가격 6억 원 초과: 45%\n\n이 비율은 1주택자에게 적용되는 특례입니다.'}

{'market_ratio': '2025년 주택 공시가격에 대한 공정시장가액비율은 1주택자에 한해 공시가격에 따라 다음과 같이 적용됩니다:\n- 공시가격 3억 원 이하: 43%\n- 공시가격 3억 원 초과~6억 원 이하: 44%\n- 공시가격 6억 원 초과: 45%\n\n다주택자 및 법인의 경우에는 공정시장가액비율이 60%로 유지됩니다.'}

In [18]:
tax_base_calculation_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',"""
주어진 내용을 기반으로 과세표준을 계산해주세요

과세표준 계산 공식: {tax_base_equation}
공제금액: {tax_deduction}
공정시장가액비율: {market_ratio}"""),
        ('human', '사용자 주택 공시가격 정보: {query}')
    ]
)


def calculate_tax_base(state:AgentState):
    tax_base_equation = state['tax_base_equation']
    tax_deduction = state['tax_deduction']
    market_ratio = state['market_ratio']

    query = state['query']
    tax_base_calculation_chain = (
        tax_base_calculation_prompt 
        | llm
        | StrOutputParser()
    )

    tax_base = tax_base_calculation_chain.invoke({
        'tax_base_equation': tax_base_equation,
        'tax_deduction' : tax_deduction,
        'market_ratio': market_ratio,
        'query': query
    })
    print(f'tax_base=={tax_base}')
    return {'tax_base': tax_base}

In [19]:
initial_state = ({
    'query':query,
    'tax_base_equation': '과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율',
    'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원입니다. 법인 또는 법인으로 보는 단체는 6억 원, 그 외의 경우는 9억 원이 공제됩니다.',
    'market_ratio': '오늘 날짜인 2025년 7월 14일에 해당하는 주택 공시가격 공정시장가액비율은 다음과 같습니다:\n\n- 공시가격 3억 원 이하: 43%\n- 공시가격 3억 원 초과 ~ 6억 원 이하: 44%\n- 공시가격 6억 원 초과: 45%\n\n이 비율은 1주택자에게 적용되는 특례입니다.',
    'tax_base' :""""""
})

In [ ]:
calculate_tax_base(initial_state)

tax_base==주어진 정보를 바탕으로 과세표준을 계산해 보겠습니다.

1. 주택 공시가격 합계:
   - 5억 원짜리 집 1채
   - 10억 원짜리 집 1채
   - 20억 원짜리 집 1채

   총 공시가격 = 5억 + 10억 + 20억 = 35억 원

2. 공제금액:
   - 사용자가 1세대 1주택자가 아니라면, 기본 공제금액은 9억 원입니다.

3. 공정시장가액비율:
   - 5억 원짜리 집: 44%
   - 10억 원짜리 집: 45%
   - 20억 원짜리 집: 45%

   공시가격이 6억 원을 초과하는 경우 45%를 적용합니다.

4. 과세표준 계산:
   - 과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율
   - 과세표준 = (35억 원 - 9억 원) × 45%
   - 과세표준 = 26억 원 × 0.45
   - 과세표준 = 11.7억 원

따라서, 사용자는 과세표준에 따라 종합부동산세를 계산해야 합니다. 실제 세금 금액은 과세표준에 해당하는 세율을 적용하여 산출됩니다. 세율은 해당 과세표준 구간에 따라 다르게 적용되므로, 이에 대한 추가 정보가 필요합니다.
{'tax_base': '주어진 정보를 바탕으로 과세표준을 계산해 보겠습니다.\n\n1. 주택 공시가격 합계:\n   - 5억 원짜리 집 1채\n   - 10억 원짜리 집 1채\n   - 20억 원짜리 집 1채\n\n   총 공시가격 = 5억 + 10억 + 20억 = 35억 원\n\n2. 공제금액:\n   - 사용자가 1세대 1주택자가 아니라면, 기본 공제금액은 9억 원입니다.\n\n3. 공정시장가액비율:\n   - 5억 원짜리 집: 44%\n   - 10억 원짜리 집: 45%\n   - 20억 원짜리 집: 45%\n\n   공시가격이 6억 원을 초과하는 경우 45%를 적용합니다.\n\n4. 과세표준 계산:\n   - 과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율\n   - 과세표준 = (35억 원 - 9억 원) × 45%\n   - 과세표준 

In [22]:
tax_rate_calculation_prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 종합부동산세 계산 전문가입니다. 아래 문서를 참고해서 사용자의 질문에 대한 종합부동산세를 계산해주세요\n\n종합부동산세 세율:{context}'),
    ('human', '과세표준과 사용자가 소지한 주택의 수가 아래와 같을 때 종합부동산세를 계산해주세요. \n과세표준: {tax_base}\n주택 수:{query}')

])

def calculate_tax_rate(state:AgentState):
    query = state['query']
    tax_base = state['tax_base']
    context = retriever.invoke(query)
    tax_rate_chain = (
        tax_rate_calculation_prompt
        | llm
        | StrOutputParser()
    )
    tax_rate = tax_rate_chain.invoke({'context':context, 'tax_base':tax_base, 'query':query})
    print(f'tax_rate=={tax_rate}')
    return {'answer':tax_rate}

In [23]:
initial_state = {
    'tax_base':"""1. 주택 공시가격 합계:
   - 5억 원짜리 집 1채
   - 10억 원짜리 집 1채
   - 20억 원짜리 집 1채

   총 공시가격 = 5억 + 10억 + 20억 = 35억 원

2. 공제금액:
   - 사용자가 1세대 1주택자가 아니라면, 기본 공제금액은 9억 원입니다.

3. 공정시장가액비율:
   - 5억 원짜리 집: 44%
   - 10억 원짜리 집: 45%
   - 20억 원짜리 집: 45%

   공시가격이 6억 원을 초과하는 경우 45%를 적용합니다.

4. 과세표준 계산:
   - 과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율
   - 과세표준 = (35억 원 - 9억 원) × 45%
   - 과세표준 = 26억 원 × 0.45
   - 과세표준 = 11.7억 원

따라서, 사용자는 과세표준에 따라 종합부동산세를 계산해야 합니다. 실제 세금 금액은 과세표준에 해당하는 세율을 적용하여 산출됩니다. 세율은 해당 과세표준 구간에 따라 다르게 적용되므로, 이에 대한 추가 정보가 필요합니다.""",
'query':query
}

In [24]:
calculate_tax_rate(initial_state)

tax_rate==주택 공시가격 합계가 35억 원이고, 공제금액이 9억 원이며, 과세표준이 11.7억 원인 상황에서, 사용자는 3채의 주택을 소유하고 있습니다. 따라서 "납세의무자가 3주택 이상을 소유한 경우"의 세율을 적용해야 합니다.

과세표준이 11.7억 원일 때, 세율 표에서 확인해보면 이는 "6억 원 초과 12억 원 이하" 구간에 해당합니다. 해당 구간의 세율은 다음과 같습니다:

- 기본세액: 360만 원
- 6억 원을 초과하는 금액에 적용할 세율: 1천분의 10

따라서, 종합부동산세는 다음과 같이 계산됩니다:

1. 6억 원 초과분: 11.7억 원 - 6억 원 = 5.7억 원
2. 초과분에 대한 세금: 5.7억 원 × 0.001 = 570만 원
3. 총 세금: 360만 원 + 570만 원 = 930만 원

따라서, 사용자가 내야 할 종합부동산세는 930만 원입니다.


{'answer': '주택 공시가격 합계가 35억 원이고, 공제금액이 9억 원이며, 과세표준이 11.7억 원인 상황에서, 사용자는 3채의 주택을 소유하고 있습니다. 따라서 "납세의무자가 3주택 이상을 소유한 경우"의 세율을 적용해야 합니다.\n\n과세표준이 11.7억 원일 때, 세율 표에서 확인해보면 이는 "6억 원 초과 12억 원 이하" 구간에 해당합니다. 해당 구간의 세율은 다음과 같습니다:\n\n- 기본세액: 360만 원\n- 6억 원을 초과하는 금액에 적용할 세율: 1천분의 10\n\n따라서, 종합부동산세는 다음과 같이 계산됩니다:\n\n1. 6억 원 초과분: 11.7억 원 - 6억 원 = 5.7억 원\n2. 초과분에 대한 세금: 5.7억 원 × 0.001 = 570만 원\n3. 총 세금: 360만 원 + 570만 원 = 930만 원\n\n따라서, 사용자가 내야 할 종합부동산세는 930만 원입니다.'}

In [25]:
graph_builder.add_node('get_tax_base_equation', get_tax_base_equation)
graph_builder.add_node('get_tax_deduction', get_tax_deduction)
graph_builder.add_node('get_market_ratio', get_market_ratio)
graph_builder.add_node('calculate_tax_base', calculate_tax_base)
graph_builder.add_node('calculate_tax_rate', calculate_tax_rate)

NameError: name 'get_tax_deduction' is not defined